In [ ]:
from glob import glob
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# 업로드한 concrete_image.zip 파일 확인
glob('concrete_image.zip')

In [ ]:
# 이미지 폴더 생성 및 concrete_image.zip 파일 압축 풀기

if not os.path.exists('IMAGE'):
    os.mkdir('IMAGE')
    !unzip concrete_image.zip -d IMAGE

In [ ]:
# concrete_image Negative 폴더 안의 이지미 갯수
!ls -l ./IMAGE/Negative/ | grep jpg | wc -l

In [ ]:
# concrete_image Positive 폴더 안의 이지미 갯수
!ls -l ./IMAGE/Positive/ | grep jpg | wc -l

In [ ]:
# 이미지 패스 지정
path = './IMAGE/Negative/00001.jpg'

In [ ]:
# 이미지 읽어오기
gfile = tf.io.read_file(path)
image = tf.io.decode_image(gfile)

In [ ]:
# 이미지 shape 확인
image.shape

In [ ]:
# 읽어온 Negative 이미지 보기
plt.imshow(image)
plt.show()

- ImageDataGenerator 와 flow_from_directory 이용하여 이미지 데이터셋 생성, 라벨링(라벨인코딩,원핫인코딩) 한꺼번에 처리 할수 있다.
- [조건] IMAGE 폴더 있고 그 아래 각 class별 이름 폴더가 있고 class별 폴더 안에 이미지 파일 있을 경우, 아래 수행합니다.</font>
- class 폴더 이름을 label로 취급
- 이미지 읽어 메모리 올리기
- 이미지 데이터 Augmentation 가능
- class 폴더 이름 label을 One-Hot-Encoding 수행
- 이미지, 라벨 튜플 묶고, shuffle, batch 후 데이터셋 생성


In [ ]:
# Hyperparameter Tunning

num_epochs = 10
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (227, 227, 3)  # 사이즈 확인
num_classes = 2    # Postive , Negative

In [ ]:
# 이미지 증강 시킬때 사용하는 ImageDataGenerator 함수 import
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### ImageDataGenerator 와  flow_from_directory 기능
+ ImageDataGenerator : 이미지 데이터에 대해 증강(augmentation)과 rescaling, validation_split 등의 전처리 기능 수행
+ flow_from_directory : 실제 이미지 데이터 읽고 배치, 셔플하고 labeling 수행 및 데이터셋 생성

In [ ]:
# ImageDataGenerator 이용하여 이미지 증강과 전처리하기
# - 여기서는 data augmentation 하지 않음 : 주석처리했으며 필요하면 주석해체하여 사용하면 됨
# - 1개 IMAGE 폴더로 Train, Test dataset으로 나누어야 되므로 validation_split 사용해야 함
# - validation 데이터 사이즈 입력 : validation_split=0.2 --> train set : valid set = 8 : 2
# - rescaling 수행

image_datagen = ImageDataGenerator(
      rescale=1. / 255,
      validation_split=0.2     # train set : valid set = 8 : 2
#       rotation_range=30,
#       width_shift_range=0.1,
#       height_shift_range=0.1,
#       shear_range=0.1,
#       zoom_range=0.1,
#       horizontal_flip=True,
#       fill_mode='nearest'
    )

In [ ]:
# ImageDataGenerator.flow_from_directory
# - 이미지 폴더 내의 데이터  읽고 배치 , 셔플하고 labeling 수행
# - 2개 Class에 대한 라벨링(라벨인코딩,원핫인코딩) 수행
# - subset = 'training' --> training_generator 생성
# - subset = 'validation' --> test_generator 생성
# - 수행결과, IMAGE 폴더로 Train 4,800건, Test 1,200건 dataset 만듬

# IMAGE 포더 밑에 .ipynb_checkpoints 폴더 있을경우 폴데 삭제
!rm -rf ./IMAGE/.ipynb_checkpoints

# categorical: Returns a one-hot encoded vector whose length equals the number of classes found
# Practical rule: Choose binary for a two-class sigmoid model
# Choose categorical for three or more classes-or for two classes when you prefer softmax and one-hot targets
training_generator = image_datagen.flow_from_directory(
    './IMAGE',
    batch_size=batch_size,        # batch_size = 32
    target_size=(227, 227),       # 원하는 출력 사이즈 입력. 데이터 생성후 (227, 227, 3) 변환. 디폴트 color_mode='rgb', 3 채널
    class_mode = 'categorical',   # binary 혹은 categorical
    shuffle = True,
    subset = 'training'           # training 혹은 validation. ImageDataGenerator의 validation_split 사용하므로 subset 지정해야
    )

test_generator = image_datagen.flow_from_directory(
    './IMAGE',
    batch_size=batch_size,
    target_size=(227, 227),       # 원하는 출력 사이즈 입력. 데이터 생성후 (227, 227, 3) 변환. 디폴트 color_mode='rgb', 3 채널
    class_mode = 'categorical',   # binary 혹은 categorical
    shuffle = False,
    subset = 'validation'         # training 혹은 validation. ImageDataGenerator의 validation_split 사용하므로 subset 지정해야
    )

In [ ]:
# Train, test 건수 확인
# batch_size : 32
len(training_generator) * batch_size , len(test_generator) * batch_size

In [ ]:
# class 이름 및 번호 매핑 확인
print(training_generator.class_indices)

In [ ]:
# 데이터셋에서 1개 가져오기
batch_img, batch_label = next(iter(training_generator))
print(batch_img.shape)    # 32개의 사진 이미지
print(batch_label.shape)  # 32개의 사진에 대한 원핫인코딩된 라벨

In [ ]:
print('True Value : ', batch_label[0])  # 32개의 사진 이미지중 첫번째 사진의 라벨
plt.imshow(batch_img[0])   # 32개의 사진 이미지중 첫번째 사진 이미지
plt.show()

In [ ]:
# Functional API 모델 정의

inputs = tf.keras.layers.Input(input_shape)

net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(inputs)  # 227 X 227 X 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32, (3, 3), padding='SAME')(net)  # 227 X 227 X 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)  # 113 X 113 X 32
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(net)  # 113 X 113 X 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(64, (3, 3), padding='SAME')(net)  # 113 X 113 X 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(net)  # 56 X 56 X 64
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Flatten()(net)  # 200,704
net = tf.keras.layers.Dense(512)(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Dropout(dropout_rate)(net)
net = tf.keras.layers.Dense(num_classes)(net)
net = tf.keras.layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [ ]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function
              metrics=['accuracy'])  # Metrics / Accuracy

In [ ]:
model.summary()

In [ ]:
# callback : EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

# ModelCheckpoint
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

# ReduceLROnPlateau : val_loss가 2번 이상 감소되지 않으면 lr * factor = lr 새로운 lr로 변경해서 학습 진행
lrReducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
# ImageDataGenerator 와 flow_from_directory 이용하여 데이터 만들었을때 아래와 같이 학습
# val_accuracy 50% 정도 나옴
# num_epochs = 10
# batch_size = 32

history = model.fit(
    training_generator,
    validation_data = test_generator,
    epochs=10,
    batch_size = batch_size,
    callbacks=[es, checkpoint, lrReducer]
)

In [ ]:
history.history.keys()

In [ ]:
# 성능 그래프
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

In [ ]:
# test_generator 샘플 데이터 가져오기
# 배치 사이즈 32 확인
batch_img, batch_label = next(iter(test_generator))
print(batch_img.shape)
print(batch_label.shape)

In [ ]:
# 32개 Test 샘플 이미지 그려보고 예측해 보기

i = 1
plt.figure(figsize=(16, 30))
for img, label in list(zip(batch_img, batch_label)):
    pred = model.predict(img.reshape(-1,227,227,3))
    pred_t = np.argmax(pred)
    plt.subplot(8, 4, i)
    plt.title(f'True Value:{np.argmax(label)}, Pred Value: {pred_t}')
    plt.imshow(img)
    i = i + 1